<a href="https://colab.research.google.com/github/bgcarcellar/UStAC-Codes/blob/main/Grade_Submission_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.8 MB/s eta 0:00:00


In [10]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import google_colab_selenium as gs
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

#Web scraping

In [24]:
# Step 1: Define the login URL and payload
login_url = 'https://crs.upd.edu.ph/'
target_url = 'https://crs.upd.edu.ph/grade_submission/'
payload = {
    'username': 'dge',
    'password': 'updgecrs1937',
    # Include any additional fields required by the login form
}

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Set up ChromeDriver
service = Service('/usr/local/bin/chromedriver')
driver = gs.Chrome()

wait = WebDriverWait(driver, 30)
driver.get(login_url)
wait.until(EC.element_to_be_clickable((By.NAME, "txt_login"))).send_keys(payload['username'])

password_field = driver.find_element(By.NAME, 'pwd_password')
password_field.send_keys(payload['password'])

# Submit the login form
password_field.send_keys(Keys.RETURN)
print("Logged in")

# Navigate to the target URL
driver.get(target_url)

<IPython.core.display.Javascript object>

Logged in


In [26]:
# ================================
# Scraping Grade Submission
# ================================

wait = WebDriverWait(driver, 20)

# ------------------------------------------------
# STEP 1: Read Academic Term options (safe read)
# ------------------------------------------------
term_select = wait.until(
    EC.presence_of_element_located((By.ID, "opt_term"))
)

options = term_select.find_elements(By.TAG_NAME, "option")

print("\nAvailable Academic Terms:")
for idx, opt in enumerate(options, start=1):
    print(f"{idx}. {opt.text.strip()}")

# ------------------------------------------------
# STEP 2: User selects term
# ------------------------------------------------
while True:
    try:
        choice = int(input("\nSelect Academic Term (enter number): "))
        if 1 <= choice <= len(options):
            break
        else:
            print("Invalid choice.")
    except ValueError:
        print("Enter a number.")

selected_value = options[choice - 1].get_attribute("value")
selected_text = options[choice - 1].text.strip()

# ------------------------------------------------
# STEP 3: JS-only selection (NO Selenium element reuse)
# ------------------------------------------------
driver.execute_script(
    """
    var select = document.getElementById('opt_term');
    select.value = arguments[0];
    select.dispatchEvent(new Event('change', { bubbles: true }));
    """,
    selected_value
)

# ------------------------------------------------
# STEP 4: Wait until page reflects new term
# ------------------------------------------------
wait.until(
    lambda d: d.find_element(By.ID, "opt_term")
              .find_element(By.CSS_SELECTOR, "option:checked")
              .get_attribute("value") == selected_value
)

current_text = driver.find_element(
    By.CSS_SELECTOR,
    "#opt_term option:checked"
).text.strip()

print("\nLoaded Academic Term:", current_text)

if current_text != selected_text:
    raise RuntimeError("Academic term did not load correctly")

print("✔ Academic term page loaded successfully")


# ------------------------------------------------
# STEP 5: Scraping the grade submissions
# ------------------------------------------------
grade_table = wait.until(
    EC.presence_of_element_located((By.ID, "tbl_classes"))
)

rows = grade_table.find_elements(By.XPATH, ".//tbody/tr")
print(f"Found {len(rows)} class rows")

subject_data = []

for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")

    # Defensive: skip malformed rows
    if len(cells) < 5:
        continue

    no = cells[0].text.strip()
    class_code = cells[1].text.strip()
    class_name = cells[2].text.strip()
    sched_instr = cells[3].text.strip()

    # Extract hyperlink from last column
    try:
        link_elem = cells[4].find_element(By.TAG_NAME, "a")
        grade_link = link_elem.get_attribute("href")
    except:
        grade_link = None

    subject_data.append([
        no,
        class_code,
        class_name,
        sched_instr,
        grade_link
    ])

print("Number of columns:", len(subject_data[0]))

headers = [
    "No",
    "Class Code",
    "Class",
    "Schedule / Instructor",
    "Grade Link",
]

import pandas as pd
df = pd.DataFrame(subject_data, columns=headers)
df[["Schedule", "Instructor"]] = (
    df["Schedule / Instructor"]
    .str.split("\n", n=1, expand=True)
)


Available Academic Terms:
1. First Semester AY 2024-2025
2. Second Semester AY 2024-2025
3. Midyear Term 2025
4. First Semester AY 2025-2026

Select Academic Term (enter number): 3

Loaded Academic Term: Midyear Term 2025
✔ Academic term page loaded successfully
Found 9 class rows
Number of columns: 5


In [27]:
import re

def parse_grade(grade_text):
    g = grade_text.strip().upper()

    result = {
        "category": None,
        "numeric": None
    }

    # No grade yet
    if g == "":
        result["category"] = "no_grade"
        return result

    # Drops
    if g in ["DRP", "DROP", "D"]:
        result["category"] = "drop"
        return result

    # Pass / Fail marks
    if g == "P":
        result["category"] = "pass"
        return result

    if g == "F":
        result["category"] = "fail"
        return result

    # INC cases
    if g.startswith("INC"):
        match = re.search(r"\(([\d.]+)\)", g)
        if match:
            val = float(match.group(1))
            result["numeric"] = val
            if val <= 3.0:
                result["category"] = "inc_completed_pass"
            else:
                result["category"] = "inc_completed_fail"
        else:
            result["category"] = "inc_uncompleted"
        return result

    # 4.00 cases
    if g.startswith("4.00"):
        match = re.search(r"\(([\d.]+)\)", g)
        if match:
            val = float(match.group(1))
            result["numeric"] = val   # 3.00 or 5.00
            if val <= 3.0:
                result["category"] = "four_removed_pass"
            else:
                result["category"] = "four_removed_fail"
        else:
            result["numeric"] = 4.00
            result["category"] = "four_unremoved"
        return result

    # Normal numeric grades
    try:
        val = float(g)
        result["numeric"] = val
        if val <= 3.0:
            result["category"] = "pass"
        else:
            result["category"] = "fail"
        return result
    except:
        result["category"] = "unknown"
        return result

#Exporting Grades

In [21]:
import json
from collections import defaultdict

def scrape_assign_grades_page(driver, wait, url, class_code):
    driver.get(url)

    wait.until(lambda d: d.find_elements(By.CSS_SELECTOR, "table tr"))
    rows = driver.find_elements(By.CSS_SELECTOR, "table tr")

    counts = defaultdict(int)
    numeric_grades = []
    grades_json = []

    for row in rows[1:]:  # skip header
        cols = row.find_elements(By.TAG_NAME, "td")

        # --- HARD FILTER: must be a real student row ---
        if len(cols) < 6:
            continue

        student_no = cols[1].text.strip()
        if not student_no.isdigit():
            continue  # skip non-student rows

        name = cols[2].text.strip()
        program = cols[3].text.strip()

        # --- CRS grade column ---
        grade_text = cols[-2].text.strip()

        parsed = parse_grade(grade_text)
        category = parsed["category"]
        numeric = parsed["numeric"]

        counts["total"] += 1
        counts[category] += 1

        # include only valid numeric grades
        if numeric is not None and 1.0 <= numeric <= 5.0:
            numeric_grades.append(numeric)

        grades_json.append({
            "student_no": student_no,
            "name": name,
            "program": program,
            "grade_raw": grade_text,
            "category": category,
            "numeric_grade": numeric
        })

    avg = (
        round(sum(numeric_grades) / len(numeric_grades), 4)
        if numeric_grades else None
    )

    # --- WRITE JSON FILE ---
    json_filename = f"grades_{class_code}.json"
    with open(json_filename, "w", encoding="utf-8") as f:
        json.dump(grades_json, f, indent=2)

    return {
        "total": counts["total"],
        "no_grade": counts["no_grade"],
        "drop": counts["drop"],

        "inc_uncompleted": counts["inc_uncompleted"],
        "inc_completed_pass": counts["inc_completed_pass"],
        "inc_completed_fail": counts["inc_completed_fail"],

        "four_unremoved": counts["four_unremoved"],
        "four_removed_pass": counts["four_removed_pass"],
        "four_removed_fail": counts["four_removed_fail"],

        "pass": counts["pass"],
        "fail": counts["fail"]
    }, avg

In [28]:
import time

df["Total Enrolled"] = 0
df["No Grade Yet"] = 0
df["Dropped"] = 0

df["INC (Uncompleted)"] = 0
df["INC (Completed Pass)"] = 0
df["INC (Completed Fail)"] = 0

df["4.00 (Unremoved)"] = 0
df["4.00 → Pass"] = 0
df["4.00 → Fail"] = 0

df["Passing"] = 0
df["Failing"] = 0
df["Average Grade"] = None

wait = WebDriverWait(driver, 20)

for idx, row in df.iterrows():
    link = row["Grade Link"]
    class_code = row["Class Code"]
    class_name_final = row["Class"].translate(str.maketrans({
    "/": "_",
    " ": "_",
    "\n": "_"})) + "_" + current_text.replace(" ", "_")

    if not link:
        continue

    print(f"Processing class {class_name_final}")

    counts, avg = scrape_assign_grades_page(
        driver=driver,
        wait=wait,
        url=link,
        class_code=class_name_final
    )

    df.at[idx, "Total Enrolled"] = counts["total"]
    df.at[idx, "No Grade Yet"] = counts["no_grade"]
    df.at[idx, "Dropped"] = counts["drop"]

    df.at[idx, "INC (Uncompleted)"] = counts["inc_uncompleted"]
    df.at[idx, "INC (Completed Pass)"] = counts["inc_completed_pass"]
    df.at[idx, "INC (Completed Fail)"] = counts["inc_completed_fail"]

    df.at[idx, "4.00 (Unremoved)"] = counts["four_unremoved"]
    df.at[idx, "4.00 → Pass"] = counts["four_removed_pass"]
    df.at[idx, "4.00 → Fail"] = counts["four_removed_fail"]

    df.at[idx, "Passing"] = counts["pass"]
    df.at[idx, "Failing"] = counts["fail"]
    df.at[idx, "Average Grade"] = avg

    time.sleep(1)  # be polite to CRS


Processing class GE_12_HFB_Midyear_Term_2025
Processing class GE_192_CAGI_Midyear_Term_2025
Processing class GE_192_JMM_Midyear_Term_2025
Processing class GE_193_2A_Midyear_Term_2025
Processing class GE_193_2B_Midyear_Term_2025
Processing class GE_193_2C_Midyear_Term_2025
Processing class GE_200_AMT_Midyear_Term_2025
Processing class GE_300_JAP-2_Midyear_Term_2025
Processing class GE_300_RBR-1_Midyear_Term_2025


#Savin the Files in GDrive

In [14]:
def get_or_create_folder(folder_name, parent_id):
    query = (
        f"title='{folder_name}' and "
        f"mimeType='application/vnd.google-apps.folder' and "
        f"'{parent_id}' in parents and trashed=false"
    )

    file_list = drive.ListFile({'q': query}).GetList()
    if file_list:
        return file_list[0]['id']

    folder = drive.CreateFile({
        'title': folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [{'id': parent_id}]
    })
    folder.Upload()
    return folder['id']

def upload_file(local_path, parent_id):
    f = drive.CreateFile({
        'title': local_path.split('/')[-1],
        'parents': [{'id': parent_id}]
    })
    f.SetContentFile(local_path)
    f.Upload()

In [29]:
import os

term_name_safe = current_text.replace("/", "-")

summary_file = f"master_summary_{term_name_safe}.xlsx"
df.to_excel(summary_file, index=False)

PARENT_FOLDER_ID = "1EWvsmqk3ThIEGyM_HonVjRNm5HxKaReV"

term_folder_id = get_or_create_folder(
    folder_name=current_text,
    parent_id=PARENT_FOLDER_ID
)

print("Uploading files to:", current_text)

upload_file(summary_file, term_folder_id)

for fname in os.listdir():
    if fname.startswith("grades_") and fname.endswith(".json"):
        upload_file(fname, term_folder_id)

print("Successfully uploaded all files to Google Drive")
for fname in os.listdir():
    if fname.startswith("grades_") and fname.endswith(".json"):
        os.remove(fname)
        print(f"Deleted {fname}")
if os.path.exists(summary_file):
    os.remove(summary_file)
    print(f"Deleted {summary_file}")

Uploading files to: Midyear Term 2025
Successfully uploaded all files to Google Drive
Deleted grades_GE_192_JMM_Midyear_Term_2025.json
Deleted grades_GE_193_2B_Midyear_Term_2025.json
Deleted grades_GE_200_AMT_Midyear_Term_2025.json
Deleted grades_GE_193_2A_Midyear_Term_2025.json
Deleted grades_GE_300_JAP-2_Midyear_Term_2025.json
Deleted grades_GE_12_HFB_Midyear_Term_2025.json
Deleted grades_GE_192_CAGI_Midyear_Term_2025.json
Deleted grades_GE_300_RBR-1_Midyear_Term_2025.json
Deleted grades_GE_193_2C_Midyear_Term_2025.json
Deleted master_summary_Midyear Term 2025.xlsx
